### Manage access rule table

In [ ]:
-- create access rule table
-- drop table if it exists 
if object_id('dbo.accessrule', 'u') is not null
    drop table dbo.accessrule;
go

create table dbo.accessrule (
    ruleid int not null,
    rolename varchar(128) not null, 
    tablename varchar(260) not null, 
    filterexpression varchar(max) not null, 
    isactive bit not null,
    created  datetime2(6) not null,
    updated  datetime2(6) 
);
go

In [2]:
-- select rule table
select * from dbo.accessrule
order by 1

In [6]:
-- update rule table
update dbo.accessrule
set rolename = 'fabric_user'
where ruleid = 3;

In [7]:
-- insert values into accessrule tables
insert into dbo.accessrule(ruleid, rolename, tablename, filterexpression, isactive, created)
values(4,'fabric_admin','materialized_t.orders', '1=1', 1, sysdatetime())

In [3]:
-- select from processing queue table
select * from dbo.rlsprocessingqueue


In [4]:
-- check predicate function definition

declare @schemaname varchar(128) = N'materialized_t';
declare @functionname varchar(128) = N'fn_predicate_orders'; 

select 
m.object_id, m.definition, o.name
--m.definition
from sys.sql_modules as m
inner join sys.objects as o on m.object_id = o.object_id
inner join sys.schemas as s on o.schema_id = s.schema_id
where s.name = @schemaname  
  and o.name = @functionname 
  and o.type = 'IF';   

In [5]:
-- on/off security policy in a table
alter security policy materialized_t.acct_doc_hdr_rls_policy with (state = off);
alter security policy materialized_t.orders_rls_policy with (state = off);

### Manage Roles

In [ ]:
-- create role
create role fabric_admin;
create role fabric_user;

-- add group/user to a role 
-- adding fabric engineer group to fabric admin role ( you can add a group or user)
alter role fabric_admin add member fabric_engineer               
alter role [fabric_user] add member [tmathew@oakwoodsys.com]     

In [8]:
-- select user/group in the database principal

select a.name rolename, a.type roletype, c.name as principal_name
from sys.database_principals a
left join sys.database_role_members b on a.principal_id = b.role_principal_id
left join sys.database_principals c on b.member_principal_id = c.principal_id
where c.name In ( 'smorris@oakwoodsys.com'
,'brett.moore.com#EXT#@osgi.onmicrosoft.com', 'tmathew@oakwoodsys.com')
and a.type = 'R'

### RLS Stored Procedure log table

In [ ]:
-- create RLS porcessing log table
-- drop table if it exists 
if object_id('dbo.rlsprocessingqueue', 'u') is not null
    drop table dbo.rlsprocessingqueue;
go

create table dbo.rlsprocessingqueue (
    schemaname varchar(128) not null,
    tablename varchar(128) not null,
    isprocessed bit not null,
    lasterror varchar(max) null,
    lastprocesseddate datetime2(6) null
);
go

In [ ]:
-- select from log table 
select * from dbo.rlsprocessingqueue

### Tables used to validate RLS implementation

In [ ]:
select * from materialized_t.acct_doc_hdr
select * from materialized_t.orders